Consider the best way to integrate this code into the existing infrastructure, including where it should be placed in the file structure.


In [ ]:
from typing import Dict, List, Tuple
from django.db import transaction
from django.db.models import Q
import numpy as np
from .models import Student, Organization, Match

class MatchingService:
    def __init__(self, weights: Dict[str, float] = None):
        self.weights = weights or {
            'grade': 0.40,
            'soi': 0.30,
            'area': 0.15,
            'location': 0.10,
            'work_mode': 0.05
        }

    def calculate_match_score(
        self,
        student: Student,
        org: Organization
    ) -> Tuple[float, Dict]:
        """Calculate match score and return score with breakdown."""

        # Calculate individual component scores
        grade_score = student.normalized_grade
        soi_score = student.soi_score / 25.0 if student.soi_score else 0

        # Calculate area alignment (practice areas match)
        area_score = self._calculate_area_alignment(
            student.soi_analysis.get('areas', []) if student.soi_analysis else [],
            org.practice_areas
        )

        # Location match
        location_score = 1.0 if student.location_preference == org.location else 0.0

        # Work mode preference match
        work_score = 1.0 if student.work_preference == org.work_mode else 0.0

        # Calculate weighted score
        component_scores = {
            'grade': grade_score * self.weights['grade'],
            'soi': soi_score * self.weights['soi'],
            'area': area_score * self.weights['area'],
            'location': location_score * self.weights['location'],
            'work_mode': work_score * self.weights['work_mode']
        }

        total_score = sum(component_scores.values())

        return total_score, component_scores

    def _calculate_area_alignment(
        self,
        student_areas: List[str],
        org_areas: List[str]
    ) -> float:
        """Calculate how well student's interests align with org's practice areas."""
        if not student_areas or not org_areas:
            return 0.0

        # Convert to sets for intersection
        student_set = set(student_areas)
        org_set = set(org_areas)

        # Calculate Jaccard similarity
        intersection = len(student_set.intersection(org_set))
        union = len(student_set.union(org_set))

        return intersection / union if union > 0 else 0.0

    @transaction.atomic
    def run_matching(self) -> List[Match]:
        """Execute the matching algorithm and create Match objects."""

        # Get all unmatched students and organizations
        students = Student.objects.filter(
            ~Q(match__isnull=False)  # No existing matches
        )
        organizations = Organization.objects.filter(
            capacity__gt=0
        )

        # Calculate all possible matches and scores
        potential_matches = []
        for student in students:
            for org in organizations:
                score, breakdown = self.calculate_match_score(student, org)
                potential_matches.append({
                    'student': student,
                    'organization': org,
                    'score': score,
                    'breakdown': breakdown
                })

        # Sort by score (highest first)
        potential_matches.sort(key=lambda x: x['score'], reverse=True)

        # Track assignments
        assigned_students = set()
        org_capacities = {org.id: org.capacity for org in organizations}
        matches = []

        # Create matches greedily
        for match in potential_matches:
            student = match['student']
            org = match['organization']

            # Skip if student already matched or org at capacity
            if (student.id in assigned_students or
                org_capacities[org.id] <= 0):
                continue

            # Create match
            new_match = Match.objects.create(
                student=student,
                organization=org,
                score=match['score'],
                score_breakdown=match['breakdown']
            )

            matches.append(new_match)
            assigned_students.add(student.id)
            org_capacities[org.id] -= 1

        return matches

    def preview_matches(self) -> List[Dict]:
        """Generate potential matches without saving them."""
        students = Student.objects.filter(
            ~Q(match__isnull=False)
        )
        organizations = Organization.objects.filter(
            capacity__gt=0
        )

        previews = []
        for student in students:
            student_matches = []
            for org in organizations:
                score, breakdown = self.calculate_match_score(student, org)
                student_matches.append({
                    'student_id': student.id,
                    'student_name': student.name,
                    'organization_id': org.id,
                    'organization_name': org.name,
                    'score': score,
                    'breakdown': breakdown
                })

            # Sort org matches for this student
            student_matches.sort(key=lambda x: x['score'], reverse=True)
            previews.extend(student_matches[:3])  # Top 3 matches per student

        return previews

    @transaction.atomic
    def manual_override(
        self,
        student_id: int,
        organization_id: int
    ) -> Match:
        """Create a manual match override."""
        student = Student.objects.get(id=student_id)
        org = Organization.objects.get(id=organization_id)

        # Calculate score for record-keeping
        score, breakdown = self.calculate_match_score(student, org)

        # Create match with override flag
        match = Match.objects.create(
            student=student,
            organization=org,
            score=score,
            score_breakdown=breakdown,
            is_manual_override=True
        )

        return match

In [ ]:
from rest_framework import viewsets, status
from rest_framework.decorators import action
from rest_framework.response import Response
from rest_framework.permissions import IsAdminUser
from .services import MatchingService
from .serializers import MatchSerializer, MatchPreviewSerializer
from .models import Match

class MatchingViewSet(viewsets.ModelViewSet):
    queryset = Match.objects.all()
    serializer_class = MatchSerializer
    permission_classes = [IsAdminUser]

    @action(detail=False, methods=['post'])
    def run_matching(self, request):
        """Run the matching algorithm."""
        weights = request.data.get('weights', None)

        try:
            matching_service = MatchingService(weights=weights)
            matches = matching_service.run_matching()

            return Response({
                'matches_created': len(matches),
                'matches': MatchSerializer(matches, many=True).data
            })
        except Exception as e:
            return Response({
                'error': str(e)
            }, status=status.HTTP_400_BAD_REQUEST)

    @action(detail=False, methods=['get'])
    def preview(self, request):
        """Preview potential matches."""
        weights = request.query_params.get('weights', None)

        matching_service = MatchingService(weights=weights)
        previews = matching_service.preview_matches()

        return Response(MatchPreviewSerializer(previews, many=True).data)

    @action(detail=False, methods=['post'])
    def manual_override(self, request):
        """Create a manual match override."""
        student_id = request.data.get('student_id')
        organization_id = request.data.get('organization_id')

        if not student_id or not organization_id:
            return Response({
                'error': 'Both student_id and organization_id are required'
            }, status=status.HTTP_400_BAD_REQUEST)

        try:
            matching_service = MatchingService()
            match = matching_service.manual_override(student_id, organization_id)

            return Response(MatchSerializer(match).data)
        except Exception as e:
            return Response({
                'error': str(e)
            }, status=status.HTTP_400_BAD_REQUEST)

In [ ]:
"""
Enhanced matching algorithm implementation based on weighted scoring.
Can run alongside existing matching logic.
"""
from typing import Dict, List, Set, Tuple
from django.db import transaction
from django.db.models import F
from ..models import StudentProfile, OrganizationProfile, MatchingRound, Match

class EnhancedMatchingService:
    def __init__(self, config: Dict = None):
        self.config = config or {
            'grade_weight': 0.40,
            'soi_weight': 0.30,
            'area_weight': 0.15,
            'location_weight': 0.10,
            'workpref_weight': 0.05
        }

    def calculate_fit_score(self, student: StudentProfile, org: OrganizationProfile) -> Tuple[float, Dict]:
        """Calculate compatibility score between student and organization"""
        # Calculate individual components
        grade_score = self._normalize_grade(student)
        soi_score = len(student.statements_of_interest or []) / 5.0  # Normalize to 0-1
        area_score = self._calculate_area_match(student, org)
        location_score = self._calculate_location_match(student, org)
        work_pref_score = self._calculate_work_pref_match(student, org)

        # Calculate weighted score
        final_score = (
            self.config['grade_weight'] * grade_score +
            self.config['soi_weight'] * soi_score +
            self.config['area_weight'] * area_score +
            self.config['location_weight'] * location_score +
            self.config['workpref_weight'] * work_pref_score
        )

        # Store score breakdown for transparency
        breakdown = {
            'grade': grade_score,
            'soi': soi_score,
            'area': area_score,
            'location': location_score,
            'work_pref': work_pref_score
        }

        return final_score, breakdown

    def _normalize_grade(self, student: StudentProfile) -> float:
        """Normalize student grade to 0-1 scale"""
        # Implementation depends on your grade storage format
        # This is a placeholder implementation
        return 0.8  # Example normalized grade

    def _calculate_area_match(self, student: StudentProfile, org: OrganizationProfile) -> float:
        """Calculate area of interest match score"""
        student_interests = set(student.statements_of_interest or [])
        org_areas = {org.area_of_law} if org.area_of_law else set()

        if not student_interests or not org_areas:
            return 0.0

        overlap = len(student_interests.intersection(org_areas))
        return min(1.0, overlap / len(org_areas))

    def _calculate_location_match(self, student: StudentProfile, org: OrganizationProfile) -> float:
        """Calculate location preference match"""
        student_locations = set(student.location_preferences or [])
        if not student_locations:
            return 0.5  # Neutral score if no preference
        return 1.0 if org.location in student_locations else 0.0

    def _calculate_work_pref_match(self, student: StudentProfile, org: OrganizationProfile) -> float:
        """Calculate work preference match"""
        student_prefs = set(student.work_preferences or [])
        if not student_prefs:
            return 0.5  # Neutral score if no preference
        return 1.0 if org.work_mode in student_prefs else 0.0

    @transaction.atomic
    def run_matching(self, round_number: int) -> MatchingRound:
        """Run the enhanced matching algorithm"""
        # Get or create matching round
        matching_round, _ = MatchingRound.objects.get_or_create(
            round_number=round_number,
            defaults={'status': 'in_progress'}
        )

        # Get all unmatched students and organizations
        students = StudentProfile.objects.filter(is_matched=False)
        orgs = OrganizationProfile.objects.filter(
            filled_positions__lt=F('available_positions')
        )

        # Calculate all possible matches
        matches = []
        for student in students:
            for org in orgs:
                score, breakdown = self.calculate_fit_score(student, org)
                matches.append((score, student, org, breakdown))

        # Sort matches by score (highest first)
        matches.sort(reverse=True, key=lambda x: x[0])

        # Assign matches while respecting capacity
        matched_students: Set[int] = set()
        org_capacities = {org.id: org.available_positions - org.filled_positions for org in orgs}
        successful_matches = []

        for score, student, org, breakdown in matches:
            if (student.id not in matched_students and
                org_capacities.get(org.id, 0) > 0):
                # Create match
                match = Match(
                    student=student,
                    organization=org,
                    match_score=score,
                    status='PENDING',
                    notes=f"Score breakdown: {breakdown}"
                )
                successful_matches.append(match)
                matched_students.add(student.id)
                org_capacities[org.id] -= 1

        # Bulk create matches and update statistics
        if successful_matches:
            Match.objects.bulk_create(successful_matches)

            # Update matching round statistics
            matching_round.matched_count = len(successful_matches)
            matching_round.total_students = students.count()
            matching_round.status = 'completed'
            matching_round.save()

        return matching_round

def run_matching(round_number: int) -> MatchingRound:
    """
    Enhanced version of the matching algorithm that maintains compatibility
    with existing code while adding weighted scoring.
    """
    service = EnhancedMatchingService()
    return service.run_matching(round_number)

In [ ]:
from typing import List, Dict, Tuple, Set
from django.db import transaction
from django.db.models import F
from ..models import Student, Organization, Match

class MatchingService:
    """Service for matching students to organizations"""

    DEFAULT_WEIGHTS = {
        'grade_weight': 0.40,
        'soi_weight': 0.30,
        'area_weight': 0.15,
        'location_weight': 0.10,
        'workpref_weight': 0.05
    }

    def __init__(self, config: Dict = None):
        self.config = config or self.DEFAULT_WEIGHTS

    def compute_fit_score(self, student: Student, org: Organization) -> float:
        """Calculate fit score between a student and organization"""
        w1 = self.config['grade_weight']
        w2 = self.config['soi_weight']
        w3 = self.config['area_weight']
        w4 = self.config['location_weight']
        w5 = self.config['workpref_weight']

        # Normalize grade (assuming 0-4 scale)
        grade_val = student.grade_average / 4.0 if student.grade_average else 0

        # Statement of interest score (assuming 0-25 scale)
        soi_val = student.soi_score / 25.0 if student.soi_score else 0

        # Area alignment (0-2 scale)
        area_val = self._compute_area_alignment(student, org) / 2.0

        # Location match (0-2 scale)
        loc_val = self._compute_location_match(student, org) / 2.0

        # Work preference match (0-2 scale)
        work_val = self._compute_workpref_match(student, org) / 2.0

        return (
            w1 * grade_val +
            w2 * soi_val +
            w3 * area_val +
            w4 * loc_val +
            w5 * work_val
        )

    def _compute_area_alignment(self, student: Student, org: Organization) -> int:
        """
        Calculate area alignment score (0-2)
        0: No overlap
        1: Partial overlap
        2: Strong overlap
        """
        student_areas = set(student.areas_of_interest or [])
        org_areas = set(org.practice_areas or [])

        if not student_areas or not org_areas:
            return 0

        overlap = len(student_areas & org_areas)
        if overlap >= 2:
            return 2
        return 1 if overlap == 1 else 0

    def _compute_location_match(self, student: Student, org: Organization) -> int:
        """
        Calculate location match score (0-2)
        0: Mismatch
        1: Partial match (e.g., same city different area)
        2: Perfect match
        """
        student_locs = set(student.location_preferences or [])
        org_loc = org.location

        if not student_locs or not org_loc:
            return 0

        if org_loc in student_locs:
            return 2
        # Could add partial matching logic here
        return 0

    def _compute_workpref_match(self, student: Student, org: Organization) -> int:
        """
        Calculate work preference match score (0-2)
        0: Mismatch
        1: Partial match
        2: Perfect match
        """
        if not student.work_preferences or not org.work_type:
            return 0

        if org.work_type in student.work_preferences:
            return 2
        return 0

    def build_pair_scores(
        self,
        students: List[Student],
        orgs: List[Organization]
    ) -> List[Tuple[float, Student, Organization]]:
        """Build and sort all valid student-org pairs with scores"""
        pair_scores = []

        for student in students:
            for org in orgs:
                # Skip if hard constraints aren't met
                if org.requires_local and not self._compute_location_match(student, org):
                    continue

                score = self.compute_fit_score(student, org)
                if score >= 0.2:  # Minimum threshold
                    pair_scores.append((score, student, org))

        return sorted(pair_scores, key=lambda x: x[0], reverse=True)

    @transaction.atomic
    def run_matching(
        self,
        students: List[Student],
        orgs: List[Organization]
    ) -> List[Match]:
        """
        Run the matching algorithm and create Match objects
        Returns list of created matches
        """
        pair_scores = self.build_pair_scores(students, orgs)
        matched_students: Set[int] = set()
        org_capacities: Dict[int, int] = {
            org.id: org.available_positions for org in orgs
        }
        matches = []

        for score, student, org in pair_scores:
            if (student.id not in matched_students and
                org_capacities[org.id] > 0):

                match = Match.objects.create(
                    student=student,
                    organization=org,
                    match_score=score,
                    status='PENDING',
                    is_manual_override=False
                )
                matches.append(match)

                matched_students.add(student.id)
                org_capacities[org.id] -= 1

        return matches

In [ ]:
from rest_framework import viewsets, status, permissions
from rest_framework.decorators import action
from rest_framework.response import Response
from rest_framework.pagination import PageNumberPagination
from django.db.models import Q
from celery.result import AsyncResult
from .models import Student, Organization, Match
from .tasks import run_matching_task, analyze_all_statements
from .serializers import (
    MatchSerializer,
    MatchConfigSerializer,
    MatchResultsSerializer,
    TaskStatusSerializer
)

class MatchResultsPagination(PageNumberPagination):
    page_size = 50
    page_size_query_param = 'page_size'
    max_page_size = 200

class MatchingViewSet(viewsets.ViewSet):
    permission_classes = [permissions.IsAuthenticated]
    pagination_class = MatchResultsPagination

    @action(detail=False, methods=['post'])
    def run_matching(self, request):
        """
        Trigger matching process with optional configuration

        Expected payload:
        {
            "weights": {
                "grade_weight": 0.40,
                "soi_weight": 0.30,
                "area_weight": 0.15,
                "location_weight": 0.10,
                "workpref_weight": 0.05
            },
            "round_id": "optional-round-identifier"
        }
        """
        serializer = MatchConfigSerializer(data=request.data)
        if not serializer.is_valid():
            return Response(serializer.errors, status=status.HTTP_400_BAD_REQUEST)

        # Validate there are students and orgs to match
        student_count = Student.objects.filter(
            is_active=True,
            is_matched=False
        ).count()
        org_count = Organization.objects.filter(
            is_active=True,
            available_positions__gt=0
        ).count()

        if student_count == 0 or org_count == 0:
            return Response({
                'error': 'No eligible students or organizations found',
                'student_count': student_count,
                'organization_count': org_count
            }, status=status.HTTP_400_BAD_REQUEST)

        # Start matching task
        task = run_matching_task.delay(
            config=serializer.validated_data.get('weights'),
            round_id=serializer.validated_data.get('round_id')
        )

        return Response({
            'task_id': task.id,
            'status': 'Matching process started',
            'student_count': student_count,
            'organization_count': org_count
        })

    @action(detail=False, methods=['get'])
    def results(self, request):
        """
        Get paginated matching results with optional filters

        Query parameters:
        - status: Filter by match status (pending/approved/rejected)
        - search: Search by student or organization name
        - round_id: Filter by matching round
        - min_score: Filter by minimum match score
        """
        queryset = Match.objects.select_related(
            'student',
            'organization'
        ).order_by('-match_score', '-created_at')

        # Apply filters
        status_filter = request.query_params.get('status')
        if status_filter:
            queryset = queryset.filter(status=status_filter.upper())

        search_query = request.query_params.get('search')
        if search_query:
            queryset = queryset.filter(
                Q(student__name__icontains=search_query) |
                Q(organization__name__icontains=search_query)
            )

        round_id = request.query_params.get('round_id')
        if round_id:
            queryset = queryset.filter(round_id=round_id)

        min_score = request.query_params.get('min_score')
        if min_score and min_score.replace('.', '').isdigit():
            queryset = queryset.filter(match_score__gte=float(min_score))

        # Paginate results
        paginator = self.pagination_class()
        page = paginator.paginate_queryset(queryset, request)

        serializer = MatchResultsSerializer(page, many=True)
        return paginator.get_paginated_response(serializer.data)

    @action(detail=False, methods=['post'])
    def analyze_statements(self, request):
        """
        Trigger Statement of Interest analysis

        This will only analyze statements that haven't been scored yet
        """
        # Check if there are unanalyzed statements
        unanalyzed_count = Student.objects.filter(
            soi_score__isnull=True,
            statement_of_interest__isnull=False
        ).count()

        if unanalyzed_count == 0:
            return Response({
                'message': 'No unanalyzed statements found'
            })

        task = analyze_all_statements.delay()

        return Response({
            'task_id': task.id,
            'status': 'Analysis started',
            'statements_to_analyze': unanalyzed_count
        })

    @action(detail=False, methods=['get'])
    def task_status(self, request):
        """Check status of a background task"""
        task_id = request.query_params.get('task_id')
        if not task_id:
            return Response({
                'error': 'task_id parameter is required'
            }, status=status.HTTP_400_BAD_REQUEST)

        result = AsyncResult(task_id)
        serializer = TaskStatusSerializer({
            'task_id': task_id,
            'status': result.status,
            'result': result.result if result.ready() else None
        })
        return Response(serializer.data)

    @action(detail=True, methods=['post'])
    def override_match(self, request, pk=None):
        """
        Override an existing match or create a manual match

        Expected payload:
        {
            "student_id": "id",
            "organization_id": "id",
            "notes": "Optional override reason"
        }
        """
        try:
            match = Match.objects.get(pk=pk)

            # Create new manual match
            new_match = Match.objects.create(
                student_id=request.data['student_id'],
                organization_id=request.data['organization_id'],
                is_manual_override=True,
                override_notes=request.data.get('notes'),
                override_by=request.user,
                status='APPROVED'
            )

            # Mark old match as overridden
            match.status = 'OVERRIDDEN'
            match.override_notes = request.data.get('notes')
            match.override_by = request.user
            match.save()

            serializer = MatchSerializer(new_match)
            return Response(serializer.data)

        except Match.DoesNotExist:
            return Response({
                'error': 'Match not found'
            }, status=status.HTTP_404_NOT_FOUND)
        except (KeyError, ValueError) as e:
            return Response({
                'error': str(e)
            }, status=status.HTTP_400_BAD_REQUEST)

In [ ]:
# Start matching process
POST /api/matching/run_matching/
{
    "weights": {
        "grade_weight": 0.40,
        "soi_weight": 0.30,
        "area_weight": 0.15,
        "location_weight": 0.10,
        "workpref_weight": 0.05
    }
}

# Get results with filters
GET /api/matching/results/?status=pending&min_score=0.8&page=1

# Check task status
GET /api/matching/task_status/?task_id=<task_id>

# Override a match
POST /api/matching/123/override_match/
{
    "student_id": "456",
    "organization_id": "789",
    "notes": "Manual override due to special circumstances"
}

In [ ]:
export interface MatchingConfig {
  weights: {
    grade_weight: number;
    soi_weight: number;
    area_weight: number;
    location_weight: number;
    workpref_weight: number;
  };
  round_id?: string;
}

export interface MatchResult {
  id: string;
  student: {
    id: string;
    name: string;
    grade_average: number;
    soi_score: number;
  };
  organization: {
    id: string;
    name: string;
    available_positions: number;
  };
  score: number;
  score_breakdown: {
    grade: number;
    soi: number;
    area: number;
    location: number;
    workpref: number;
  };
  status: 'PENDING' | 'APPROVED' | 'REJECTED' | 'OVERRIDDEN';
  created_at: string;
  is_manual_override: boolean;
  override_notes?: string;
}

export interface TaskResponse {
  task_id: string;
  status: string;
  student_count?: number;
  organization_count?: number;
}

In [ ]:
import React, { useState } from 'react';
import { useForm, Controller } from 'react-hook-form';
import { useMutation, useQueryClient } from '@tanstack/react-query';
import { MatchingConfig, TaskResponse } from '../../types/matching';
import { Slider } from '../common/Slider';
import { Alert } from '../common/Alert';
import { api } from '../../utils/api';

const DEFAULT_WEIGHTS = {
  grade_weight: 0.4,
  soi_weight: 0.3,
  area_weight: 0.15,
  location_weight: 0.1,
  workpref_weight: 0.05
};

export const MatchingConfiguration: React.FC = () => {
  const queryClient = useQueryClient();
  const [taskId, setTaskId] = useState<string | null>(null);

  const { control, handleSubmit, watch, reset } = useForm<MatchingConfig>({
    defaultValues: {
      weights: DEFAULT_WEIGHTS
    }
  });

  const weights = watch('weights');
  const totalWeight = Object.values(weights).reduce((sum, w) => sum + w, 0);
  const isValidTotal = Math.abs(totalWeight - 1) <= 0.01;

  const matchingMutation = useMutation<TaskResponse, Error, MatchingConfig>({
    mutationFn: async (config) => {
      const response = await api.post('/matching/run_matching/', config);
      return response.data;
    },
    onSuccess: (data) => {
      setTaskId(data.task_id);
      queryClient.invalidateQueries(['matches']);
    }
  });

  const onSubmit = (data: MatchingConfig) => {
    if (!isValidTotal) {
      return;
    }
    matchingMutation.mutate(data);
  };

  const onReset = () => {
    reset({ weights: DEFAULT_WEIGHTS });
  };

  return (
    <div className="bg-white rounded-lg shadow p-6">
      <div className="flex justify-between items-center mb-6">
        <h2 className="text-xl font-semibold">Matching Configuration</h2>
        <button
          onClick={onReset}
          className="text-sm text-gray-600 hover:text-gray-800"
          type="button"
        >
          Reset to Defaults
        </button>
      </div>

      {matchingMutation.isError && (
        <Alert
          type="error"
          message={matchingMutation.error.message}
          className="mb-4"
        />
      )}

      {taskId && (
        <Alert
          type="success"
          message={`Matching process started (Task ID: ${taskId})`}
          className="mb-4"
        />
      )}

      <form onSubmit={handleSubmit(onSubmit)} className="space-y-4">
        {Object.entries(DEFAULT_WEIGHTS).map(([key, _]) => (
          <div key={key} className="space-y-2">
            <label className="flex justify-between text-sm font-medium text-gray-700">
              <span>{key.replace('_', ' ').toUpperCase()}</span>
              <span className="text-gray-500">
                {weights[key as keyof typeof weights].toFixed(2)}
              </span>
            </label>
            <Controller
              name={`weights.${key}`}
              control={control}
              render={({ field }) => (
                <Slider
                  min={0}
                  max={1}
                  step={0.05}
                  value={field.value}
                  onChange={field.onChange}
                  disabled={matchingMutation.isLoading}
                />
              )}
            />
          </div>
        ))}

        <div className={`flex items-center justify-between p-3 rounded ${
          isValidTotal ? 'bg-green-50 text-green-700' : 'bg-red-50 text-red-700'
        }`}>
          <span className="font-medium">
            Total Weight: {totalWeight.toFixed(2)}
          </span>
          {!isValidTotal && (
            <span className="text-sm">Should equal 1.0</span>
          )}
        </div>

        <div className="flex justify-end space-x-3 pt-4">
          <button
            type="submit"
            disabled={matchingMutation.isLoading || !isValidTotal}
            className={`
              px-4 py-2 rounded-md text-white font-medium
              ${isValidTotal
                ? 'bg-blue-600 hover:bg-blue-700'
                : 'bg-gray-400 cursor-not-allowed'
              }
            `}
          >
            {matchingMutation.isLoading ? (
              <span className="flex items-center">
                <svg className="animate-spin -ml-1 mr-3 h-5 w-5 text-white" xmlns="http://www.w3.org/2000/svg" fill="none" viewBox="0 0 24 24">
                  <circle className="opacity-25" cx="12" cy="12" r="10" stroke="currentColor" strokeWidth="4"></circle>
                  <path className="opacity-75" fill="currentColor" d="M4 12a8 8 0 018-8V0C5.373 0 0 5.373 0 12h4zm2 5.291A7.962 7.962 0 014 12H0c0 3.042 1.135 5.824 3 7.938l3-2.647z"></path>
                </svg>
                Running Matching...
              </span>
            ) : (
              'Run Matching'
            )}
          </button>
        </div>
      </form>
    </div>
  );
};

In [ ]:
import React, { useState } from 'react';
import { useQuery } from '@tanstack/react-query';
import { MatchResult } from '../../types/matching';
import { api } from '../../utils/api';
import { Pagination } from '../common/Pagination';
import { SearchInput } from '../common/SearchInput';
import { StatusBadge } from '../common/StatusBadge';
import { ScoreBreakdown } from './ScoreBreakdown';

interface MatchResultsFilters {
  status?: string;
  search?: string;
  page: number;
  min_score?: number;
}

export const MatchResults: React.FC = () => {
  const [filters, setFilters] = useState<MatchResultsFilters>({
    page: 1
  });

  const {
    data,
    isLoading,
    error,
    isFetching
  } = useQuery(
    ['matches', filters],
    async () => {
      const params = new URLSearchParams();
      Object.entries(filters).forEach(([key, value]) => {
        if (value) params.append(key, value.toString());
      });
      const response = await api.get(`/matching/results/?${params.toString()}`);
      return response.data;
    },
    {
      keepPreviousData: true
    }
  );

  const handleFilterChange = (newFilters: Partial<MatchResultsFilters>) => {
    setFilters(prev => ({ ...prev, ...newFilters, page: 1 }));
  };

  if (error) {
    return (
      <div className="bg-red-50 p-4 rounded-md">
        <p className="text-red-700">Error loading matches: {error.message}</p>
      </div>
    );
  }

  return (
    <div className="bg-white rounded-lg shadow">
      <div className="p-6 border-b border-gray-200">
        <div className="flex justify-between items-center mb-6">
          <h2 className="text-xl font-semibold">Match Results</h2>
          <div className="flex space-x-4">
            <SearchInput
              value={filters.search || ''}
              onChange={(value) => handleFilterChange({ search: value })}
              placeholder="Search students or organizations..."
            />
            <select
              value={filters.status || ''}
              onChange={(e) => handleFilterChange({ status: e.target.value })}
              className="rounded-md border-gray-300"
            >
              <option value="">All Statuses</option>
              <option value="PENDING">Pending</option>
              <option value="APPROVED">Approved</option>
              <option value="REJECTED">Rejected</option>
              <option value="OVERRIDDEN">Overridden</option>
            </select>
          </div>
        </div>

        {isLoading ? (
          <div className="flex justify-center py-8">
            <div className="animate-spin rounded-full h-8 w-8 border-b-2 border-blue-600"></div>
          </div>
        ) : (
          <>
            <div className="overflow-x-auto">
              <table className="min-w-full divide-y divide-gray-200">
                <thead className="bg-gray-50">
                  <tr>
                    <th className="px-6 py-3 text-left text-xs font-medium text-gray-500 uppercase tracking-wider">
                      Student
                    </th>
                    <th className="px-6 py-3 text-left text-xs font-medium text-gray-500 uppercase tracking-wider">
                      Organization
                    </th>
                    <th className="px-6 py-3 text-left text-xs font-medium text-gray-500 uppercase tracking-wider">
                      Score
                    </th>
                    <th className="px-6 py-3 text-left text-xs font-medium text-gray-500 uppercase tracking-wider">
                      Status
                    </th>
                    <th className="px-6 py-3 text-left text-xs font-medium text-gray-500 uppercase tracking-wider">
                      Score Breakdown
                    </th>
                  </tr>
                </thead>
                <tbody className="bg-white divide-y divide-gray-200">
                  {data?.results.map((match: MatchResult) => (
                    <tr key={match.id} className="hover:bg-gray-50">
                      <td className="px-6 py-4 whitespace-nowrap">
                        <div className="text-sm font-medium text-gray-900">
                          {match.student.name}
                        </div>
                        <div className="text-sm text-gray-500">
                          Grade: {match.student.grade_average.toFixed(2)}
                        </div>
                      </td>
                      <td className="px-6 py-4 whitespace-nowrap">
                        <div className="text-sm text-gray-900">
                          {match.organization.name}
                        </div>
                        <div className="text-sm text-gray-500">
                          Positions: {match.organization.available_positions}
                        </div>
                      </td>
                      <td className="px-6 py-4 whitespace-nowrap">
                        <div className="text-sm font-medium text-gray-900">
                          {match.score.toFixed(3)}
                        </div>
                      </td>
                      <td className="px-6 py-4 whitespace-nowrap">
                        <StatusBadge status={match.status} />
                        {match.is_manual_override && (
                          <span className="ml-2 text-xs text-gray-500">
                            (Manual Override)
                          </span>
                        )}
                      </td>
                      <td className="px-6 py-4">
                        <ScoreBreakdown breakdown={match.score_breakdown} />
                      </td>
                    </tr>
                  ))}
                </tbody>
              </table>
            </div>

            <div className="mt-4">
              <Pagination
                currentPage={filters.page}
                totalPages={Math.ceil(data?.count / 50)}
                onPageChange={(page) => setFilters(prev => ({ ...prev, page }))}
              />
            </div>
          </>
        )}
      </div>
    </div>
  );
};

In [ ]:
import React from 'react';
import { Tooltip } from '../common/Tooltip';

interface ScoreBreakdownProps {
  breakdown: {
    grade: number;
    soi: number;
    area: number;
    location: number;
    workpref: number;
  };
}

const COLORS = {
  grade: 'bg-blue-500',
  soi: 'bg-green-500',
  area: 'bg-yellow-500',
  location: 'bg-purple-500',
  workpref: 'bg-red-500'
};

export const ScoreBreakdown: React.FC<ScoreBreakdownProps> = ({ breakdown }) => {
  return (
    <div className="flex h-2 rounded-full overflow-hidden">
      {Object.entries(breakdown).map(([key, value]) => (
        <Tooltip
          key={key}
          content={`${key}: ${(value * 100).toFixed(1)}%`}
        >
          <div
            className={`${COLORS[key as keyof typeof COLORS]}`}
            style={{ width: `${value * 100}%` }}
          />
        </Tooltip>
      ))}
    </div>
  );
};